In [74]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import re
import nltk
from sklearn.metrics import confusion_matrix,accuracy_score
import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [8]:
df=pd.read_csv("amazon_data2.csv")

In [36]:
df1=df[['review_text','review_rating']]
df1['Sentiment']=np.where(df1['review_rating']>3,1,0)

C:\Users\INMAANN2\AppData\Local\Temp\ipykernel_9744\2906889517.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Sentiment']=np.where(df1['review_rating']>3,1,0)


In [60]:
# df1[df1['Sentiment']==0]
df1.dropna(axis=0,inplace=True)
df1.reset_index(drop=True,inplace=True)

C:\Users\INMAANN2\AppData\Local\Temp\ipykernel_9744\264679981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(axis=0,inplace=True)


In [61]:
df1

,review_text,review_rating,Sentiment
0,i bought this laptop at the price of 80499 bef...,5.0,1
1,The laptop is a beast. Tremendous performance ...,5.0,1
2,I changed many settings and locked my fps at 6...,4.0,1
3,The product is excellent.,5.0,1
4,"Bought HP Victus 16-0351AX for 81,990 INR on 2...",4.0,1
...,...,...,...
334,"After the updates things were pretty smooth, t...",4.0,1
335,While gaming this laptop is touching 100 degre...,3.0,0
336,Don't spend this much money. Better to buy Xbo...,2.0,0
337,Beast machiery but make sure that u must keep ...,5.0,1


### Data Cleaning

In [52]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\INMAANN2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [54]:
ps=PorterStemmer()
all_stopwords=stopwords.words('english')
all_stopwords.remove('not')

In [59]:
df1['review_text'].iloc[191]

"Battery won't last long, not more than 1.5hr. if you are heavy user, then constant power source be connected. Graphics are good. Sound is ok. Light weight. Good speed.Check your requirements before buying, rtx 3050 graphics could have been better, it's GTX 1650."

In [63]:
corpus=[]
for i in range(0,df1.shape[0]):
#     print(i)
    review=re.sub('[^a-zA-Z]',' ',df1['review_text'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in set(all_stopwords)]
    review=' '.join(review)
    corpus.append(review)

### Data Transformation

In [23]:
# cvFile='c1_BoW_Sentiment_Model.pkl'
# cv=pickle.load(open(cvFile,"rb"))
cv=CountVectorizer(max_features=1420)

In [65]:
X=cv.fit_transform(corpus).toarray()
y=df1.iloc[:,-1].values

In [66]:
bow_path='c1_BoW_Sentiment_Model.pkl'
pickle.dump(cv,open(bow_path,'wb'))

In [68]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

### Model Fitting

In [70]:
classifier=GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB()

In [72]:
joblib.dump(classifier,"c2_Classifer_Sentiment_Model")

['c2_Classifer_Sentiment_Model']

In [73]:
y_pred=classifier.predict(X_test)

cm=confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[10  0]
 [ 3 55]]


0.9558823529411765